In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent, MultipleObjectsError


import ipywidgets as ipw
from IPython.display import display, clear_output

# Install and/or update plugins

In [ ]:
def on_setup_plugins(b):
    with setup_plugin_output:
        clear_output()
        ! pip install --user -e ./plugins/
        ! reentry scan -r aiida

setup_plugin_output = ipw.Output()
btn_setup_plugins = ipw.Button(description="Install spm plugins")
btn_setup_plugins.on_click(on_setup_plugins)
display(btn_setup_plugins, setup_plugin_output)

In [ ]:
def on_list_plugins(b):
    with list_plugin_output:
        ! verdi calculation plugins

list_plugin_output = ipw.Output()
btn_list_plugins = ipw.Button(description="List all plugins")
btn_list_plugins.on_click(on_list_plugins)
display(btn_list_plugins, list_plugin_output)

# Setup codes

In [ ]:
## -------------------------------------------------------
## List computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True})
all_computers = qb.all()

computer_selection = [comp[0].get_name() for comp in all_computers]

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer",
                             options=['']+computer_selection)

def show_codes(c):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True}, project=['name'], tag='computer')
    qb.append(Code, project=['*'], has_computer='computer')
    with code_output:
        clear_output()
        print("Codes on " + drop_computer.value + ":")
        for e in qb.all():
            if e[0] == drop_computer.value:
                print(e[1])
                
code_output = ipw.Output()

drop_computer.observe(show_codes, names='value')

show_codes(0)

display(ipw.VBox([drop_computer, code_output]))

# STM codes

Set the path of the remote computer to install the external scripts to. Needs to be a location available to the calculation nodes (e.g. not `/project/` on Piz-Daint). 

In [ ]:
def retrive_scripts(b):
    
    global version_id, code_dir_path
    
    commit_hash = ! git ls-remote "https://github.com/eimrek/atomistic_tools.git"
    version_id = commit_hash[0][:7]

    code_dir_name = "at_"+version_id

    code_remote_path = install_path_inp.value

    code_dir_path = code_remote_path+"/"+code_dir_name

    computer = Computer.get(drop_computer.value)

    with retrieve_output:
        
        clear_output()

        # ssh key needs to be set
        ! ssh {computer.hostname} "wget https://github.com/eimrek/atomistic_tools/archive/master.zip -O {code_remote_path}/{code_dir_name}.zip"
        ! ssh {computer.hostname} "cd {code_remote_path} && unzip -o {code_dir_name}.zip && rm -rf {code_dir_name} {code_dir_name}.zip && mv atomistic_tools-master {code_dir_name}"
        ! ssh {computer.hostname} "cd {code_remote_path}/{code_dir_name} && chmod u+x *"

code_dir_path = None
version_id = None

retrieve_output = ipw.Output()

retrieve_button = ipw.Button(description="Retrieve scripts")

retrieve_button.on_click(retrive_scripts)

install_path_inp = ipw.Text(description="Install path:", value="/users/username/aiida-soft", layout=layout, style=style)

display(install_path_inp, retrieve_button, retrieve_output)


#### Scanning tunnelling microscopy and spectroscopy images

For the STM script, `python 3` instance with `mpi4py` and `ase` is needed.

Please provide the correct bash commands to make this available (this text will be prepended to each slurm script).

In [ ]:
stm_prepend_text = ipw.Textarea(description="Prepend text:",
             layout = {'width': '70%'},
             value =
                "module load cray-python/3.6.1.1\n" +
                "export PYTHONPATH=$PYTHONPATH:\"/users/keimre/soft/ase\"\n"
            )

display(stm_prepend_text)

In [ ]:
def setup_stm():
    
    # Eval mol. orbitals
    # Python with MPI support and ASE
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += stm_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = "py_stm_"+version_id

    code_full_name = code_label + "@" + drop_computer.value

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(Computer.get(drop_computer.value), code_dir_path + "/stm_sts_from_wfn.py"))
        code.label = code_label
        code.description = "Python code to calculate STM and STS"
        code.set_input_plugin_name("spm.stm")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_stm_setup_click(b):
    with stm_setup_out:
        clear_output()
        setup_stm()
    
stm_setup_out = ipw.Output()

stm_setup_btn = ipw.Button(description="Setup stm")
stm_setup_btn.on_click(on_stm_setup_click)

display(stm_setup_btn, stm_setup_out)

#### Orbital overlap

`python 3` instance with `mpi4py` and `ase` is needed.

Please provide the correct bash commands to make this available (this text will be prepended to each slurm script).

In [ ]:
overlap_prepend_text = ipw.Textarea(description="Prepend text:",
             layout = {'width': '70%'},
             value =
                "module load cray-python/3.6.1.1\n" +
                "export PYTHONPATH=$PYTHONPATH:\"/users/keimre/soft/ase\"\n"
            )

display(overlap_prepend_text)

In [ ]:
def setup_overlap():
    
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += overlap_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = "py_overlap_"+version_id

    code_full_name = code_label + "@" + drop_computer.value

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(Computer.get(drop_computer.value), code_dir_path + "/overlap_from_wfns.py"))
        code.label = code_label
        code.description = "Python code to evaluate overlaps between orbitals"
        code.set_input_plugin_name("spm.overlap")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_overlap_setup_click(b):
    with overlap_setup_out:
        clear_output()
        setup_overlap()
    
overlap_setup_out = ipw.Output()

overlap_setup_btn = ipw.Button(description="Setup overlap")
overlap_setup_btn.on_click(on_overlap_setup_click)

display(overlap_setup_btn, overlap_setup_out)

# AFM codes

Set the path of the remote computer to install the external scripts to. Needs to be a location available to the calculation nodes (e.g. not `/project/` on Piz-Daint). 

In [ ]:
afm_pp_code_dir_path = None
afm_2pp_code_dir_path = None
sha_pp = "b5f22ca55ea329c8d49aa37338eab0f8f8e21583"
sha_2pp = "a5f77cc74b238b7ed07ba9a1e8228e70231289de"

def retrive_afm_scripts(b):
    
    global afm_pp_code_dir_path, afm_2pp_code_dir_path

    code_dir_name_pp = "ProbeParticleModel_pp"
    code_dir_name_2pp = "ProbeParticleModel_2pp"

    code_remote_path = afm_install_path.value

    afm_pp_code_dir_path = code_remote_path+"/"+code_dir_name_pp
    afm_2pp_code_dir_path = code_remote_path+"/"+code_dir_name_2pp

    computer = Computer.get(drop_computer.value)
    
    with afm_retrieve_output:
        
        clear_output()

        # ssh key needs to be set
        ! ssh {computer.hostname} "wget https://github.com/ProkopHapala/ProbeParticleModel/archive/{sha_pp}.zip -O {code_remote_path}/{code_dir_name_pp}.zip"
        ! ssh {computer.hostname} "cd {code_remote_path} && unzip -o {code_dir_name_pp}.zip && rm -rf {code_dir_name_pp} {code_dir_name_pp}.zip && mv ProbeParticleModel-{sha_pp} {code_dir_name_pp}"
        
        ! ssh {computer.hostname} "wget https://github.com/ProkopHapala/ProbeParticleModel/archive/{sha_2pp}.zip -O {code_remote_path}/{code_dir_name_2pp}.zip"
        ! ssh {computer.hostname} "cd {code_remote_path} && unzip -o {code_dir_name_2pp}.zip && rm -rf {code_dir_name_2pp} {code_dir_name_2pp}.zip && mv ProbeParticleModel-{sha_2pp} {code_dir_name_2pp}"
        
        # create a custom bash scripts
        ! scp "./afm/run_afm.sh" {computer.hostname}:{afm_pp_code_dir_path}
        ! scp "./afm/run_afm.sh" {computer.hostname}:{afm_2pp_code_dir_path}
            
        ! ssh {computer.hostname} "chmod -R 755 {afm_pp_code_dir_path}"
        ! ssh {computer.hostname} "chmod -R 755 {afm_2pp_code_dir_path}"

afm_retrieve_output = ipw.Output()

afm_retrieve_button = ipw.Button(description="Retrieve scripts")

afm_retrieve_button.on_click(retrive_afm_scripts)

afm_install_path = ipw.Text(description="Install path:", value="/users/username/aiida-soft", layout=layout, style=style)

display(afm_install_path, afm_retrieve_button, afm_retrieve_output)


For the AFM codes, `python 2` instance with `matplotlib` is needed.

Please provide the correct bash commands to make this available (this text will be prepended to each slurm script).

In [ ]:
afm_prepend_text = ipw.Textarea(description="Prepend text:",
             layout = {'width': '70%'},
             value =
                "export PATH='/users/keimre/soft/miniconda3/envs/py27/bin':$PATH\n"
            )

display(afm_prepend_text)

In [ ]:
def setup_afm_code(label, code_dir_path, description, sha):
    
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += afm_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = label+sha[:6]

    code_full_name = code_label + "@" + drop_computer.value

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(Computer.get(drop_computer.value), code_dir_path + "/run_afm.sh"))
        code.label = code_label
        code.description = description
        code.set_input_plugin_name("spm.afm")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_afm_setup_click(b):
    with afm_setup_out:
        clear_output()
        setup_afm_code("py_afm_pp_", afm_pp_code_dir_path, "Python code for ProbeParticle AFM simulation", sha_pp)
        setup_afm_code("py_afm_2pp_", afm_2pp_code_dir_path, "Python code for 2 point ProbeParticle AFM simulation", sha_2pp)
    
afm_setup_out = ipw.Output()

afm_setup_btn = ipw.Button(description="Setup afm codes")
afm_setup_btn.on_click(on_afm_setup_click)

display(afm_setup_btn, afm_setup_out)

# HR-STM codes

Set the path of the remote computer to install the external scripts to. Needs to be a location available to the calculation nodes (e.g. not `/project/` on Piz-Daint). 

In [ ]:
def retrieve_hr_scripts(b): 
    # Global: Version number (hash) + File directory
    global hr_version_id, hr_code_dir_path
    
    # Get version id from hash of HEAD
    commit_hash = ! git ls-remote "https://github.com/hifabian/HR-STM.git"
    hr_version_id = commit_hash[0][:7]
    
    code_dir_name = "HR_STM_at_"+hr_version_id
    code_remote_path = hr_install_path_inp.value
    hr_code_dir_path = code_remote_path+"/"+code_dir_name

    computer = Computer.get(drop_computer.value)

    with hr_retrieve_output:
        clear_output()
        
        # ssh key needs to be set
        ! ssh {computer.hostname} "wget https://github.com/hifabian/HR-STM/archive/master.zip -O {code_remote_path}/{code_dir_name}.zip"
        ! ssh {computer.hostname} "cd {code_remote_path} && unzip -o {code_dir_name}.zip && rm -rf {code_dir_name} {code_dir_name}.zip && mv HR-STM-master {code_dir_name}"
        ! ssh {computer.hostname} "cd {code_remote_path}/{code_dir_name} && chmod u+x run.py"

hr_code_dir_path = None
hr_version_id = None

hr_retrieve_output = ipw.Output()

hr_retrieve_button = ipw.Button(description="Retrieve scripts")

hr_retrieve_button.on_click(retrieve_hr_scripts)

hr_install_path_inp = ipw.Text(description="Install path:", value="/users/username/aiida-soft", layout=layout, style=style)

display(hr_install_path_inp, hr_retrieve_button, hr_retrieve_output)

For the HR-STM codes, `python 3` instance with `mpi4py`  and `gcc >=7.0` is needed.

Please provide the correct bash commands to make this available (this text will be prepended to each slurm script).

In [ ]:
hrstm_prepend_text = ipw.Textarea(description="Prepend text:",
             layout = {'width': '70%'},
             value =
                "module load gcc/7.1.0\n" +
                "module load cray-python/3.6.1.1\n"
            )

display(hrstm_prepend_text)

In [ ]:
def setup_hrstm():
    
    # Python with MPI support
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += hrstm_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = "py_hrstm_"+hr_version_id

    code_full_name = code_label + "@" + drop_computer.value

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(Computer.get(drop_computer.value), hr_code_dir_path+"/run.py"))
        code.label = code_label
        code.description = "Python code to calculate HR-STM"
        code.set_input_plugin_name("spm.hrstm")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_hrstm_setup_click(b):
    with hrstm_setup_out:
        clear_output()
        setup_hrstm()
    
hrstm_setup_out = ipw.Output()

hrstm_setup_btn = ipw.Button(description="Setup hrstm")
hrstm_setup_btn.on_click(on_hrstm_setup_click)

display(hrstm_setup_btn, hrstm_setup_out)